In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# Get Data

In [2]:
import urllib.request
import json

def retrieve(url):
    with urllib.request.urlopen(url) as response:
        html = response.read().decode("utf-8")
        data = json.loads(html)
    return data

In [3]:
import time
import datetime


def timestampToDatetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp)


def strTimeToDatetime(strTime):
    return datetime.datetime.strptime(strTime, "%Y-%m-%d %H:%M:%S")


def strTimeToTimestamp(strTime):
    return strTimeToDatetime(strTime).timestamp()

In [4]:
def queryPoloniex(beginDatetime, endDatetime, currencyPair, period):
    beginTimestamp, endTimestamp = beginDatetime.timestamp(), endDatetime.timestamp()
#     url = 'https://poloniex.com/public?command=returnTradeHistory&currencyPair={pair}&start={begin}&end={end}'
    url = 'https://poloniex.com/public?command=returnChartData&currencyPair={pair}&start={begin}&end={end}&period={period}'
    return retrieve(url.format(pair=currencyPair, begin=beginTimestamp, end=endTimestamp, period=period))

In [5]:
from dateutil.rrule import rrule, MONTHLY 
from time import sleep

def retrieveLongWindowPoloniex(beginTime, endTime, currencyPair, period):
    beginDT = strTimeToDatetime(beginTime)
    endDT = strTimeToDatetime(endTime)
    data = queryPoloniex(beginDT, endDT, currencyPair, period)
    return data
        

In [6]:
def renameDateToTimestamp(df):
    return df.rename(columns={'date': 'timestamp'})

In [7]:
def poloniexDataToDf(data):
    df = pd.DataFrame(data)
    df = renameDateToTimestamp(df)
    df['date'] = df['timestamp'].apply(timestampToDatetime)
    return df

# Download

In [8]:
startDateTime, endDateTime = '2014-01-01 00:00:00', '2017-12-31 23:59:59'
startDate, endDate = startDateTime[:10], endDateTime[:10]
interval, intervalStr = 900, '5min'
cPairs = [
    'USDT_BCH', 'BTC_BCH', 
    'BTC_XEM', 
    'USDT_STR', 'BTC_STR',
    'BTC_DASH',
    'USDT_ETC', 'BTC_ETC',
    'BTC_LSK'
]

In [10]:
for cPair in cPairs:
    print(cPair, end='\t')
    data = retrieveLongWindowPoloniex(startDateTime, endDateTime, cPair, interval)
    df = poloniexDataToDf(data)
    fileName = '{cPair}_{intervalStr}_{startDate}_{endDate}.tsv'.format(**locals())
    path = 'data/poloniex/{intervalStr}/{fileName}'.format(**locals())
    df.to_csv(path, index=False, sep='\t')
    print('done')

USDT_BCH	done
BTC_BCH	done
BTC_XEM	done
USDT_STR	done
BTC_STR	done
BTC_DASH	done
USDT_ETC	done
BTC_ETC	done
BTC_LSK	done
